# Union de df de actividades estandar en uno solo
---
En este notebook vamos a unir todos los dataframe estandar en uno solo que reuna la información de todas las actividades. Antes de juntarlos en un solo dataframe, debemos establecer la ventana de tiempo que vamos a considerar para recoger las features de actividad, y explorar si en los vpl y los cuestionarios hay calificaciones -1, para explorar si las marcamos como no entregadas o como entregadas.

## Configuración del entorno y las rutas

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, concat_ws, count, avg, coalesce, to_date, from_unixtime, max
import os
from pyspark.sql.functions import to_date, lit


#  Crear sesión Spark
spark = SparkSession.builder \
    .appName("EvaluacionContinuaIP") \
    .getOrCreate()

#  Rutas base
PATH_INTERMEDIATE_STD = "/home/carlos/Documentos/TFG/spark-workspace/data/intermediate/estandarizados"
PATH_STD_GLOBAL = "/home/carlos/Documentos/TFG/spark-workspace/data/intermediate/estandarizados_completos"

#Fecha de corte que separa las ventanas
FECHA_CORTE = to_date(lit("21/11/2023"), "dd/MM/yyyy")


# Crear carpeta de resultados si no existe (opcional en local)
os.makedirs(PATH_STD_GLOBAL, exist_ok=True)

# Verificación
print("✔️ Spark configurado y rutas preparadas.")


✔️ Spark configurado y rutas preparadas.


## Carga de datos

In [10]:
#Cargamos los tres datasets estandarizados y verificamos que tengan la misma estructura
df_assign = spark.read.parquet(f"{PATH_INTERMEDIATE_STD}/assign_std.parquet")
df_quiz = spark.read.parquet(f"{PATH_INTERMEDIATE_STD}/quiz_std.parquet")
df_vpl = spark.read.parquet(f"{PATH_INTERMEDIATE_STD}/vpl_std.parquet")

df_assign.printSchema()
df_quiz.printSchema()
df_vpl.printSchema()


root
 |-- userid: string (nullable = true)
 |-- actividad_id: long (nullable = true)
 |-- actividad_nombre: string (nullable = true)
 |-- actividad_codigo: string (nullable = true)
 |-- fecha_limite: date (nullable = true)
 |-- nota: double (nullable = true)
 |-- entregado: integer (nullable = true)
 |-- fecha_entrega: date (nullable = true)

root
 |-- userid: string (nullable = true)
 |-- actividad_id: long (nullable = true)
 |-- actividad_nombre: string (nullable = true)
 |-- actividad_codigo: string (nullable = true)
 |-- entregado: integer (nullable = true)
 |-- nota: double (nullable = true)
 |-- fecha_entrega: date (nullable = true)
 |-- fecha_limite: date (nullable = true)

root
 |-- userid: string (nullable = true)
 |-- actividad_id: long (nullable = true)
 |-- actividad_nombre: string (nullable = true)
 |-- actividad_codigo: string (nullable = true)
 |-- nota: double (nullable = true)
 |-- entregado: integer (nullable = true)
 |-- fecha_entrega: date (nullable = true)
 |-- fec

## Exploración de notas -1.00

Al estandarizar los datos de las assign, hemos podido ver que algunas entregas tenían puesta la calificación de -1.00. Vamos a explorar si es un fenómeno que se produce también en las otras dos tablas 

In [11]:

print("Notas -1 en VPL:")
df_vpl.filter(col("nota") == -1.0).select("actividad_id", "userid").show()

print("Notas -1 en Quiz:")
df_quiz.filter(col("nota") == -1.0).select("actividad_id", "userid").show()

print("Notas -1 en Assign:")
df_assign.filter(col("nota") == -1.0).select("actividad_id", "userid").show()


Notas -1 en VPL:
+------------+------+
|actividad_id|userid|
+------------+------+
+------------+------+

Notas -1 en Quiz:
+------------+------+
|actividad_id|userid|
+------------+------+
+------------+------+

Notas -1 en Assign:
+------------+--------------------+
|actividad_id|              userid|
+------------+--------------------+
|      107688|433f2e41671277b91...|
|      107688|d7daac23e04503587...|
|      107688|d46525fcffba400ca...|
|      107688|7414360295620f649...|
|      107688|f6be71259bf65d67b...|
|      107688|93e91045641e49c66...|
|      107688|df7ce50a753bbb53b...|
|      107688|f27dcf10007aa8bde...|
|      107688|be7ddad586818bb7a...|
|      107688|3b14a013b6f57a945...|
|      107688|3fbb342bde31891eb...|
|      107688|d6c028d3e65c04f84...|
|      107688|99ee189a85eb1a146...|
|      107688|40d647ca18ec15645...|
|      107688|76d71aa4b78891014...|
|      107688|625fcde3a86e10933...|
|      107688|d59886a5f495c3d4f...|
|      107688|93807a17ca948eafa...|
|      1076

Podemos observar, que esta calificación solo es asignada a las tareas de tipo assign, no en los cuestionarios ni vpls. Hay que decidir si se marcan como no entregadas aquellas que están marcadas con una calificación de -1. 

## Unir tablas estandarizadas para apilar toda la información de entregas de actividades en una sola.  

Vamos a unir todas las tablas estandar para quedarnos ya con toda la información de las entregas hechas por usuario en cada una de las actividades del curso, y vamos a poder ver ya todas las actividades del curso ordenadas por fecha.

In [12]:
df_actividades_raw = df_assign.unionByName(df_quiz).unionByName(df_vpl)

print("Actividades de la asignatura ordenadas por fecha límite:")
df_actividades_raw.select("actividad_id", "actividad_nombre", "actividad_codigo", "fecha_limite").distinct().orderBy("fecha_limite").show(100, truncate=False)

print("Dataframe generado con entregas de todas las actividades:")
df_actividades_raw.show(100, truncate=False)


Actividades de la asignatura ordenadas por fecha límite:
+------------+-----------------------------------------------------------------------+----------------+------------+
|actividad_id|actividad_nombre                                                       |actividad_codigo|fecha_limite|
+------------+-----------------------------------------------------------------------+----------------+------------+
|16681       |Cuestionario previo a la clase 1                                       |quiz_16681      |2023-09-13  |
|16685       |Cuestionario previo a la clase 2                                       |quiz_16685      |2023-09-14  |
|29266       |Cuestionario previo a la clase 3                                       |quiz_29266      |2023-09-19  |
|16709       |Cuestionario previo a la clase 05                                      |quiz_16709      |2023-09-21  |
|7592        |Actividad 00. Actualización del perfil en el campus virtual            |assign_7592     |2023-09-25  |
|14457 

## Filtrar tabla para quedarnos solo con información de todas las actividades hechas hasta la fecha de corte

In [13]:
df_actividades_obs = df_actividades_raw.filter(col("fecha_limite") < FECHA_CORTE)

print("Actividades de la asignatura de la ventana de observación  ordenadas por fecha límite:")
df_actividades_obs.select("actividad_id", "actividad_nombre", "actividad_codigo", "fecha_limite").distinct().orderBy("fecha_limite").show(100, truncate=False)

Actividades de la asignatura de la ventana de observación  ordenadas por fecha límite:
+------------+-------------------------------------------------------------------+----------------+------------+
|actividad_id|actividad_nombre                                                   |actividad_codigo|fecha_limite|
+------------+-------------------------------------------------------------------+----------------+------------+
|16681       |Cuestionario previo a la clase 1                                   |quiz_16681      |2023-09-13  |
|16685       |Cuestionario previo a la clase 2                                   |quiz_16685      |2023-09-14  |
|29266       |Cuestionario previo a la clase 3                                   |quiz_29266      |2023-09-19  |
|16709       |Cuestionario previo a la clase 05                                  |quiz_16709      |2023-09-21  |
|7592        |Actividad 00. Actualización del perfil en el campus virtual        |assign_7592     |2023-09-25  |
|14457   

## Convertir filas que tienen nota -1 a "no entregado" (Comentado por que ya está hecho antes de unir)

In [ ]:
# from matplotlib.pylab import f

# df_actividades_obs = df_actividades_obs.withColumn(
#     "entregado", when(col("nota") == -1.0, 0).otherwise(col("entregado"))
# )
# #Verificar que no hay notas -1 marcadas como entregadas
# n = df_actividades_obs.filter((col("nota") == -1.0) & (col("entregado") == 1)).count()
# print(f"Cantidad de notas -1 marcadas como entregadas: {n}")


Cantidad de notas -1 marcadas como entregadas: 0


## Guardamos la tabla previa al pivotado
Esta tabla puede llegar a ser de mucha utilidad, dado que sobre ella se pueden calcular muchas métricas diferentes, asi que la guardaremos como una tabla intermedia

In [14]:
df_actividades_obs.write.mode("overwrite").parquet(f"{PATH_STD_GLOBAL}/actividades_ip_std.parquet")

## Pivotamos la tabla

Finalmente, vamos a pivotar la tabla para darle el formato que necesitamos para construir el dataset

In [15]:
from pyspark.sql.functions import first

# 1. Pivot entregado
df_pivot_entregado = df_actividades_obs.groupBy("userid") \
    .pivot("actividad_codigo") \
    .agg(first("entregado"))

# 2. Pivot nota
df_pivot_nota = df_actividades_obs.groupBy("userid") \
    .pivot("actividad_codigo") \
    .agg(first("nota"))

# 3. Renombrar columnas de nota
df_pivot_nota = df_pivot_nota.select(
    [col("userid")] + [
        col(c).alias(f"{c}_nota") for c in df_pivot_nota.columns if c != "userid"
    ]
)

# 4. Unir ambos pivot
df_actividades_pivot = df_pivot_entregado.join(df_pivot_nota, on="userid", how="inner")

# 5. Reordenar columnas: [userid, act1, act1_nota, act2, act2_nota, ...]
cols_entregas = sorted([c for c in df_pivot_entregado.columns if c != "userid"])
cols_notas = [f"{c}_nota" for c in cols_entregas]

df_actividades_pivot = df_actividades_pivot.select(
    ["userid"] + [c for pair in zip(cols_entregas, cols_notas) for c in pair]
)

df_actividades_pivot.show(100, truncate=False)

df_actividades_pivot.count()
print(f"Total de usuarios en el dataframe: {df_actividades_pivot.count()}")


+----------------------------------------------------------------+-------------+------------------+-----------+----------------+----------+---------------+----------+---------------+----------+---------------+----------+---------------+----------+---------------+----------+---------------+----------+---------------+----------+---------------+----------+---------------+----------+---------------+----------+---------------+----------+---------------+-------+------------+-------+------------+-------+------------+-------+------------+
|userid                                                          |assign_107688|assign_107688_nota|assign_7592|assign_7592_nota|quiz_14457|quiz_14457_nota|quiz_16011|quiz_16011_nota|quiz_16681|quiz_16681_nota|quiz_16685|quiz_16685_nota|quiz_16709|quiz_16709_nota|quiz_16720|quiz_16720_nota|quiz_16750|quiz_16750_nota|quiz_16804|quiz_16804_nota|quiz_16862|quiz_16862_nota|quiz_16889|quiz_16889_nota|quiz_16900|quiz_16900_nota|quiz_29266|quiz_29266_nota|vpl_529|vpl

## Renombramos columnas para mejorar la legibilidad y exportamos


In [20]:
from IPython.display import display

codigo_to_alias = {
    "quiz_16681": "Clase 1",
    "quiz_16685": "Clase 2",
    "quiz_29266": "Clase 3",
    "quiz_16709": "Clase 5",
    "assign_7592": "Perfil",
    "quiz_14457": "Test Expr.",
    "quiz_16720": "Clase 6",
    "vpl_529": "Act. 02 - Elecciones",
    "quiz_16750": "Clase 10",
    "vpl_532": "Act. 03 - Catalan",
    "quiz_16804": "Clase 13",
    "vpl_534": "Act. 04 - Primos",
    "quiz_16862": "Clase 18",
    "vpl_543": "Act. 05 - Vectores",
    "quiz_16889": "Clase 22",
    "quiz_16900": "Clase 23",
    "quiz_16011": "Test Complejidad",
    "assign_107688": "Act. 07"
}

# Convertir desde Spark si aún no lo hiciste
df_actividades = df_actividades_pivot.toPandas()

# Construir nuevo nombre para columnas
nuevos_nombres = {}
for col in df_actividades.columns:
    if col == "userid":
        continue
    if col.endswith("_nota"):
        base = col.replace("_nota", "")
        alias = codigo_to_alias.get(base, base)
        nuevos_nombres[col] = f"{alias} (nota)"
    else:
        alias = codigo_to_alias.get(col, col)
        nuevos_nombres[col] = alias

# Renombrar columnas
df_actividades.rename(columns=nuevos_nombres, inplace=True)

display(df_actividades)

df_actividades.to_parquet("/home/carlos/Documentos/TFG/spark-workspace/data/datasets/actividades_ventana_observacion.parquet", index=False)



,userid,Act. 07,Act. 07 (nota),Perfil,Perfil (nota),Test Expr.,Test Expr. (nota),Test Complejidad,Test Complejidad (nota),Clase 1,...,Clase 3,Clase 3 (nota),Act. 02 - Elecciones,Act. 02 - Elecciones (nota),Act. 03 - Catalan,Act. 03 - Catalan (nota),Act. 04 - Primos,Act. 04 - Primos (nota),Act. 05 - Vectores,Act. 05 - Vectores (nota)
0,e1f1d0f48ca77093f9d66cefd325504245277db3e6c145...,1,8.50,1,2.0,1,10.00000,1,5.00000,1,...,1,7.5,1,10.0,1,7.0,1,10.0,0,NaN
1,b5de2bb5b8538b199d6b3f0ecb32daa8a9d730ccc484db...,1,6.25,1,2.0,1,10.00000,1,6.00000,1,...,1,10.0,1,10.0,1,10.0,1,10.0,1,10.00
2,90a634296aff946e9d045997d512d2b77dbc01880715c1...,1,6.00,1,2.0,1,10.00000,1,8.66667,1,...,0,NaN,1,10.0,1,10.0,1,9.0,1,5.00
3,b6b2a12e84ea8203775195ed2bb4e99c5788053782b0bd...,1,10.00,1,2.0,1,10.00000,1,7.33333,1,...,1,10.0,1,10.0,1,10.0,1,10.0,1,10.00
4,fd96e32a94a932f45eb32933d9ffeb71f4addf9153a76b...,1,0.00,1,2.0,1,6.00000,1,6.00000,1,...,0,NaN,1,10.0,1,10.0,0,NaN,1,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,a84ab1865c59659dc6eff2c5d7bd99be7d1702207963b0...,1,3.75,1,2.0,1,8.66667,1,4.66667,0,...,1,2.5,1,10.0,1,10.0,1,10.0,1,10.00
197,d46525fcffba400caa4fc21005b92fa32ed6922cf520d4...,0,-1.00,1,2.0,1,10.00000,0,NaN,0,...,1,10.0,1,0.0,0,NaN,0,NaN,0,NaN
198,571a6aae9b56541a475c2693de2e90c573534058a1063a...,1,3.00,1,2.0,0,NaN,1,6.00000,0,...,0,NaN,1,10.0,1,10.0,1,10.0,1,0.00
199,8bebcc80d3ad7e6037990a53263910dcbab9eca028145d...,1,4.00,1,2.0,1,8.66667,1,4.66667,1,...,1,7.5,1,5.0,1,7.0,1,10.0,1,8.33
